## 2021: Week 16 - The Super League

If you are a fan of football or have been following any of the latest news then you would have probably heard about the proposed new 'Super League' that was planned with a selection of European clubs. This would have involved a group of 12 teams playing in a competition each year without having to qualify or be relegated. The lack of  fair competition between other clubs has caused an uproar among fans, players, media outlets, football associations, and even the UK government! 

The 'big 6' English teams to propose the Super League were Arsenal, Chelsea, Liverpool, Manchester United, Manchester City, and Tottenham Hotspur.

One of the ideas to try and discourage the clubs from proceeding with the new league, was to threaten the English teams with being expelled from the English Premier League. The challenge this week is to try and understand how the current league table would change if these clubs were to be 'kicked out'.

### Input
The input this week is a list of all of the fixtures from the 2020/21 season (up until 19/04/2021). 

### Requirement
- Input the data
- Calculate the Total Points for each team. The points are as follows: 
    - Win - 3 Points
    - Draw - 1 Point
    - Lose - 0 Points
- Calculate the goal difference for each team. Goal difference is the difference between goals scored and goals conceded. 
- Calculate the current rank/position of each team. This is based on Total Points (high to low) and in a case of a tie then Goal Difference (high to low).
- The current league table is our first output.

- Assuming that the 'Big 6' didn't play any games this season, recalculate the league table.
- After removing the 6 clubs, how has the position changed for the remaining clubs?
- The updated league table is the second output.
- Bonus - Think about features in Tableau Prep to make this repeatable process easier!

### Output
1. Current League Table
    - 5 Fields, 20 Rows (21 including header)
    - Position
    - Team
    - Total Games Played
    - Total Points
    - Goal Difference
    
    
2. Updated League Table
    - 5 Fields, 14 Rows (15 including header)
    - Position
    - Team
    - Total Games Played
    - Total Points
    - Goal Difference

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### Input the data

In [5]:
df = pd.read_csv("./data/PL Fixtures.csv", parse_dates=["Date"])
df.head()

,Round Number,Date,Location,Home Team,Away Team,Result
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Round Number  380 non-null    int64         
 1   Date          380 non-null    datetime64[ns]
 2   Location      380 non-null    object        
 3   Home Team     380 non-null    object        
 4   Away Team     380 non-null    object        
 5   Result        316 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 17.9+ KB


In [ ]:
### Calculate the Total Points for each team. The points are as follows

In [37]:
result_null = df.loc[df["Result"].isna()].index
df = df.drop(result_null, axis=0)
df["Home Score"] = df["Result"].map(lambda x: x.split("-")[0])
df["Away Score"] = df["Result"].map(lambda x: x.split("-")[-1])
df

,Round Number,Date,Location,Home Team,Away Team,Result,Home Score,Away Score
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3,0,3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3,4,3
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2,0,2
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3,0,3
...,...,...,...,...,...,...,...,...
312,32,2021-04-17 12:30:00,St. James' Park,Newcastle,West Ham,3 - 2,3,2
314,32,2021-04-17 20:15:00,Molineux Stadium,Wolves,Sheffield Utd,1 - 0,1,0
315,32,2021-04-18 13:30:00,Emirates Stadium,Arsenal,Fulham,1 - 1,1,1
316,32,2021-04-18 16:00:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1


In [42]:
df["Home Score"] = df["Home Score"].astype(int)
df["Away Score"] = df["Away Score"].astype(int)

In [67]:
def calculate_point(row_):
    if (row_["Home Score"] - row_["Away Score"]) < 0:
        return 0, 3
    elif (row_["Home Score"] - row_["Away Score"]) > 0:
        return 3, 0
    else:
        return 1, 1

In [71]:
result = df.apply(lambda x: calculate_point(x), axis=1).apply(pd.Series)
df = pd.concat([df, result], axis=1)
df = df.rename(columns={0: "Home Point", 1: "Away Point"})
df

,Round Number,Date,Location,Home Team,Away Team,Result,Home Score,Away Score,Home Point,Away Point
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3,0,3,0,3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0,3,0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3,4,3,3,0
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2,0,2,0,3
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3,0,3,0,3
...,...,...,...,...,...,...,...,...,...,...
312,32,2021-04-17 12:30:00,St. James' Park,Newcastle,West Ham,3 - 2,3,2,3,0
314,32,2021-04-17 20:15:00,Molineux Stadium,Wolves,Sheffield Utd,1 - 0,1,0,3,0
315,32,2021-04-18 13:30:00,Emirates Stadium,Arsenal,Fulham,1 - 1,1,1,1,1
316,32,2021-04-18 16:00:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1,3,0


In [58]:
### Calculate the goal difference for each team. 
### Goal difference is the difference between goals scored and goals conceded

(0, 3)

In [136]:
home_group = df.groupby(["Home Team"])[["Home Score", "Away Score"]].sum()
home_goal_diff = home_group["Home Score"] - home_group["Away Score"]
home_goal_diff

Home Team
Arsenal            0
Aston Villa        4
Brighton          -3
Burnley           -5
Chelsea           11
Crystal Palace    -9
Everton           -3
Fulham           -15
Leeds              3
Leicester          4
Liverpool          5
Man City          22
Man Utd           13
Newcastle         -5
Sheffield Utd    -15
Southampton        1
Spurs              9
West Brom        -21
West Ham           9
Wolves             0
dtype: int32

In [137]:
away_group = df.groupby(["Away Team"])[["Home Score", "Away Score"]].sum()
away_goal_diff = away_group["Away Score"] - away_group["Home Score"]
away_goal_diff

Away Team
Arsenal            8
Aston Villa        6
Brighton          -2
Burnley          -14
Chelsea            8
Crystal Palace   -10
Everton            6
Fulham            -3
Leeds             -3
Leicester         14
Liverpool         11
Man City          22
Man Utd           16
Newcastle        -13
Sheffield Utd    -24
Southampton      -18
Spurs              8
West Brom        -10
West Ham           2
Wolves            -9
dtype: int32

In [135]:
total_goal_diff = home_goal_diff + away_goal_diff
total_goal_diff

Home Team
Arsenal            8
Aston Villa       10
Brighton          -5
Burnley          -19
Chelsea           19
Crystal Palace   -19
Everton            3
Fulham           -18
Leeds              0
Leicester         18
Liverpool         16
Man City          44
Man Utd           29
Newcastle        -18
Sheffield Utd    -39
Southampton      -17
Spurs             17
West Brom        -31
West Ham          11
Wolves            -9
dtype: int32

In [ ]:
### Calculate the current rank/position of each team. 
### This is based on Total Points (high to low) and in a case of a tie then Goal Difference (high to low).

In [143]:
home_df = df[["Round Number", "Date", "Home Team", "Home Score", "Home Point"]]
home_df.columns = ["Round Number", "Date", "Team", "Score", "Point"]

away_df = df[["Round Number", "Date", "Away Team", "Away Score", "Away Point"]]
away_df.columns = ["Round Number", "Date", "Team", "Score", "Point"]

total_df = pd.concat([home_df, away_df], axis=0)
total_df

,Round Number,Date,Team,Score,Point
0,1,2020-12-09 12:30:00,Fulham,0,0
1,1,2020-12-09 15:00:00,Crystal Palace,1,3
2,1,2020-12-09 17:30:00,Liverpool,4,3
3,1,2020-12-09 20:00:00,West Ham,0,0
4,1,2020-09-13 14:00:00,West Brom,0,0
...,...,...,...,...,...
312,32,2021-04-17 12:30:00,West Ham,2,0
314,32,2021-04-17 20:15:00,Sheffield Utd,0,0
315,32,2021-04-18 13:30:00,Fulham,1,1
316,32,2021-04-18 16:00:00,Burnley,1,0


In [147]:
current_rank = total_df.groupby(["Team"])["Point"].sum().reset_index()
current_rank["Goal Difference"] = total_goal_diff.values
current_rank

,Team,Point,Goal Difference
0,Arsenal,46,8
1,Aston Villa,44,10
2,Brighton,33,-5
3,Burnley,33,-19
4,Chelsea,54,19
5,Crystal Palace,38,-19
6,Everton,49,3
7,Fulham,27,-18
8,Leeds,46,0
9,Leicester,56,18


In [154]:
current_rank["Position"] = current_rank[["Point", "Goal Difference"]].apply(tuple, axis=1).rank(method="dense", ascending=False).astype(int)
current_rank = current_rank.sort_values(by="Position", ascending=True)
current_rank

,Team,Point,Goal Difference,Position
11,Man City,74,44,1
12,Man Utd,66,29,2
9,Leicester,56,18,3
18,West Ham,55,11,4
4,Chelsea,54,19,5
10,Liverpool,53,16,6
16,Spurs,50,17,7
6,Everton,49,3,8
0,Arsenal,46,8,9
8,Leeds,46,0,10


In [163]:
num_match = total_df.groupby(["Team"])["Score"].count().reset_index()
current_rank = current_rank.merge(num_match, how="inner", on="Team").rename(columns={"Score": "Total GamesPlayed", "Point": "Total Points"})
current_rank = current_rank.loc[:, ["Position", "Team", "Total GamesPlayed", "Total Points", "Goal Difference"]]
current_rank

,Position,Team,Total GamesPlayed,Total GamesPlayed,Total Points,Goal Difference
0,1,Man City,32,32,74,44
1,2,Man Utd,32,32,66,29
2,3,Leicester,31,31,56,18
3,4,West Ham,32,32,55,11
4,5,Chelsea,31,31,54,19
5,6,Liverpool,32,32,53,16
6,7,Spurs,32,32,50,17
7,8,Everton,31,31,49,3
8,9,Arsenal,32,32,46,8
9,10,Leeds,32,32,46,0
